In [1]:
pip install keras-tuner


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0


In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Define hyperparameters for hidden layers
    for i in range(hp.Int('num_layers', 2, 7)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), 128, 512, 32),
                               activation='relu'))

    model.add(layers.Dense(10, activation='softmax'))



    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    total_trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])

    return model, total_trainable_params


In [ ]:
! mkdir keras_tuner_results

In [ ]:
! mkdir keras_tuner_results/fashion_mnist_tuning

In [ ]:

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='keras_tuner_results',
    project_name='fashion_mnist_tuning'
)


In [ ]:
tuner.search(x_train, y_train, epochs=5, validation_split=0.1)


In [ ]:
# ... (previous code to perform hyperparameter search) ...

# Get the best hyperparameters and build the final model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model, total_trainable_params = tuner.hypermodel.build(best_hps)

print("Total trainable parameters:", total_trainable_params)


In [ ]:
history = model.fit(x_train, y_train, epochs=20, validation_split=0.1)


In [ ]:
import matplotlib.pyplot as plt
def accuracy(model):
  plt.plot(model.history['accuracy'])
  plt.plot(model.history['val_accuracy'])
  plt.title(f"{model}  Accuracy ")
  plt.ylabel("Accuracy")
  plt.xlabel("Epochs")
  plt.legend(['Train','Test'],loc='best')
  plt.show()


def loss_plot(model):
  plt.plot(model.history['loss'])
  plt.plot(model.history['val_loss'])
  plt.title(f"{model} Loss ")
  plt.ylabel("Loss")
  plt.xlabel("Epochs")
  plt.legend(['Train','Test'],loc='best')
  plt.show()


In [ ]:
accuracy(history)

In [ ]:
loss_plot(history)

In [ ]:
import matplotlib.pyplot as plt

# Retrieve the results of the tuning process
tuner_results = tuner.results_summary()




In [ ]:
import matplotlib.pyplot as plt

# Data for each trial
trial_indices = [2, 0, 3, 1, 4]
validation_accuracy = [0.886, 0.885, 0.880, 0.878, 0.871]

# Plot the bar graph
plt.figure(figsize=(8, 6))
plt.bar(trial_indices, validation_accuracy)
plt.xlabel('Trial')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy for Each Trial')
plt.xticks(trial_indices, ['Trial 2', 'Trial 0', 'Trial 3', 'Trial 1', 'Trial 4'])
plt.ylim(0.85, 0.89)  # Set the y-axis limits to better visualize the differences
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Data for each trial
trial_info = {
    'Trial 2': [288, 192, 512, 480, 288, 416],
    'Trial 0': [448, 288, 128, 128, 128, 128],
    'Trial 3': [256, 224, 160, 160, 192, 192],
    'Trial 1': [256, 160, 128, 512, 256, 352],
    'Trial 4': [288, 128, 288, 352, 192, 352]
}

# Extract the validation accuracy for each trial
validation_accuracy = [0.886, 0.885, 0.880, 0.878, 0.871]

# Plot the bar graph
plt.figure(figsize=(12, 8))
for i, (trial, units) in enumerate(trial_info.items()):
    plt.bar(range(len(units)), units, label=f'{trial}, Score: {validation_accuracy[i]}')

plt.xlabel('Hidden Layers')
plt.ylabel('Number of Hidden Units')
plt.title('Number of Hidden Units in Each Hidden Layer for Each Trial')
plt.xticks(range(len(units)), ['Layer 1', 'Layer 2', 'Layer 3', 'Layer 4', 'Layer 5', 'Layer 6'])
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
for i, trial in enumerate(best_trials):
    model, num_layers = tuner.hypermodel.build(trial.hyperparameters)
    history = model.fit(x_train, y_train, epochs=20, validation_split=0.1, verbose=0)

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label=f'Trial {i+1} (Layers: {num_layers})')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs. Epoch')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label=f'Trial {i+1} (Layers: {num_layers})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss vs. Epoch')
    plt.legend()

plt.tight_layout()
plt.show()



In [ ]:
! mkdir logs

In [ ]:
!kill 13258

In [ ]:
 %reload_ext tensorboard

In [ ]:
import datetime
# Create a TensorBoard callback with a separate log directory for each trial
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

# Perform the hyperparameter search and save the TensorBoard logs
tuner.search(x_train, y_train, epochs=5, validation_split=0.1, callbacks=[tensorboard_callback])

# Load TensorBoard in a new tab
%load_ext tensorboard
%tensorboard --logdir logs